In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, upper, translate, when
from pyspark.sql.types import IntegerType

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901"
)
conf.set(
    "spark.hadoop.fs.s3a.aws.credentials.provider",
    "com.amazonaws.auth.InstanceProfileCredentialsProvider"
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Função para remover acentos de uma coluna
def remover_acentos(df, nome_coluna):
    acentos = "áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ"
    sem_acentos = "aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC"
    return df.withColumn(nome_coluna, translate(col(nome_coluna), acentos, sem_acentos))

# =======================================================================
# 2. LEITURA DOS DADOS (ISO-8859-1)
# =======================================================================
df_raw = spark.read.option('delimiter', ';') \
                     .option('header', 'true') \
                     .option('nullValue', 'null') \
                     .option('encoding', 'ISO-8859-1') \
    .csv("s3a://bucket-raw-upa-connect-sofh/basesExternas/FebreAmarela/fa_casoshumanos_1994-2025.csv")

# =======================================================================
# 3. SELEÇÃO E LIMPEZA
# =======================================================================
colunas_desejadas = ['ID', 'UF_LPI', 'MUN_LPI', 'SEXO', 'IDADE', 'DT_IS', 'ANO_IS', 'OBITO', 'DT_OBITO']
df_limpo = df_raw.select(*colunas_desejadas)

# Substitui strings vazias por null nas colunas numéricas e converte para Integer
colunas_numericas = ['ID', 'IDADE', 'ANO_IS']
for c in colunas_numericas:
    df_limpo = df_limpo.withColumn(c, when(col(c) == "", None).otherwise(col(c)))
    df_limpo = df_limpo.withColumn(c, col(c).cast(IntegerType()))

# Remove linhas com nulos nas colunas críticas
colunas_para_verificar = ['ID', 'UF_LPI', 'MUN_LPI', 'SEXO', 'IDADE', 'DT_IS', 'ANO_IS', 'OBITO']
df_limpo = df_limpo.na.drop(subset=colunas_para_verificar)

# Filtra apenas UF_LPI = SP
df_limpo = df_limpo.filter(col("UF_LPI") == "SP")

# Converte colunas de texto para maiúsculas
colunas_maiusculas = ['UF_LPI', 'MUN_LPI', 'SEXO', 'OBITO']
for c in colunas_maiusculas:
    df_limpo = df_limpo.withColumn(c, upper(col(c)))

# =======================================================================
# 4. VALIDAÇÃO: se OBITO = SIM, DT_OBITO deve estar preenchida
# =======================================================================
# Linhas que serão removidas
linhas_invalidas = df_limpo.filter(
    (col("OBITO") == "SIM") & ((col("DT_OBITO").isNull()) | (col("DT_OBITO") == ""))
)

# Mostra os IDs dessas linhas
# É necessário coletar antes de printar no ambiente Spark
ids_invalidos = linhas_invalidas.select("ID").rdd.flatMap(lambda x: x).collect()
print(f"IDs removidos na validação OBITO: {ids_invalidos}")

# Contagem para log
linhas_removidas = linhas_invalidas.count()
print(f"Validação OBITO: {linhas_removidas} linhas removidas onde OBITO = SIM e DT_OBITO estava vazio ou nulo")

# Aplica o filtro para manter apenas linhas válidas
df_limpo = df_limpo.filter(
    (col("OBITO") != "SIM") | ((col("OBITO") == "SIM") & (col("DT_OBITO").isNotNull()) & (col("DT_OBITO") != ""))
)


colunas_com_acentos = ['MUN_LPI', 'OBITO']

for c in colunas_com_acentos:
    df_limpo = remover_acentos(df_limpo, c)


# =======================================================================
# 5. SALVANDO E RENOMEANDO O RESULTADO NO S3
# =======================================================================

# Define o caminho de saída definitivo no S3 (Diretório)
FINAL_OUTPUT_DIR = "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela"

# Define o nome do arquivo final
FINAL_FILENAME = "FebreAmarelaTratada.csv"

# Define o caminho temporário (subdiretório dentro da pasta principal)
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging"

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")
# Note que estamos usando df_limpo (seu DataFrame tratado)
df_limpo.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'ISO-8859-1') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        # Esta operação move o arquivo dentro do S3 e o renomeia.
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Dados salvos e renomeados com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")


# Encerra a sessão Spark
spark.stop()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fe2477a7-bfd7-4144-939f-63ba6d980282;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 394ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

IDs removidos na validação OBITO: [1253, 1358, 1397, 1411, 1476, 1522, 1826, 2212, 2303, 2304, 2404, 2478, 2600, 2608]
Validação OBITO: 14 linhas removidas onde OBITO = SIM e DT_OBITO estava vazio ou nulo

Escrevendo dados temporariamente em: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/_temp_staging


25/10/16 02:23:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/16 02:23:01 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Dados salvos e renomeados com sucesso para: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/FebreAmarelaTratada.csv


# 